In [ ]:
!pip install streamlit pyngrok


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("36hdXGM0bEWtOrHHs1qX6Wyj7y8_3mwuQz39RorH6MDrhVdNy")


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd

# DATASET (clean, consistent, items exist in all stores)
products = ["milk","bread","eggs","rice","oil","atta"]
stores = ["Blinkit", "Zepto", "Instamart", "JioMart"]

prices = {
    "milk":     [50, 52, 48, 55],
    "bread":    [30, 32, 28, 31],
    "eggs":     [60, 58, 62, 59],
    "rice":     [90, 85, 92, 88],
    "oil":      [120, 125, 118, 123],
    "atta":     [200, 205, 198, 210],
}

delivery = {
    "Blinkit":  [10, 10, 12, 15, 18, 20],
    "Zepto":    [12, 11, 13, 14, 19, 22],
    "Instamart":[15, 14, 16, 18, 20, 23],
    "JioMart":  [25, 24, 26, 28, 30, 35]
}

rows = []

for i, prod in enumerate(products):
    for j, store in enumerate(stores):
        rows.append({
            "product": prod,
            "store": store,
            "price": prices[prod][j],
            "delivery_time": delivery[store][i]
        })

df = pd.DataFrame(rows)

# Streamlit UI Setup
st.set_page_config(page_title="Grocery Comparison", layout="wide")

st.title("Grocery Multi-Store Comparison")
st.markdown("Enter multiple items separated by commas. Example: **milk, bread, eggs**")

query = st.text_input("Search items")

# Card Component
def card(store, price, delivery, title=""):
    st.markdown(
        f"""
        <div style='
            border:1px solid #ddd;
            padding:15px;
            border-radius:12px;
            background:black;
            box-shadow:0 2px 6px rgba(0,0,0,0.1);
            margin-bottom:10px;
        '>
            <h4 style='margin:0'>{title}</h4>
            <p style='margin:3px 0'>Store: <b>{store}</b></p>
            <p style='margin:3px 0'>Price: ₹{price}</p>
            <p style='margin:3px 0'>Delivery Time: {delivery} minutes</p>
        </div>
        """,
        unsafe_allow_html=True
    )

# SEARCH ACTION
if st.button("Search"):
    if query.strip() == "":
        st.warning("Please enter at least one item.")
    else:
        items = [x.strip().lower() for x in query.split(",")]

        st.header("Best Store For Each Item")
        st.markdown("---")

        valid_items = []

        for item in items:
            sub = df[df["product"] == item]
            if sub.empty:
                st.warning(f"{item.capitalize()} not found.")
                continue

            valid_items.append(item)

            best = sub.sort_values("price").iloc[0]

            card(
                store=best["store"],
                price=best["price"],
                delivery=best["delivery_time"],
                title=item.capitalize()
            )

        # Skip summary if no valid items
        if not valid_items:
            st.stop()


        # OVERALL CHEAPEST STORE
        st.header("Cheapest Overall Store (All Items)")
        st.markdown("---")

        summary = []

        for store in stores:
            total = 0
            del_times = []

            for item in valid_items:
                row = df[(df["store"] == store) & (df["product"] == item)]
                total += int(row["price"].values[0])
                del_times.append(int(row["delivery_time"].values[0]))

            avg_delivery = sum(del_times) / len(del_times)

            summary.append((store, total, avg_delivery))

        summary = sorted(summary, key=lambda x: x[1])

        best_store, best_total, best_avg = summary[0]

        card(
            store=best_store,
            price=best_total,
            delivery=round(best_avg),
            title=f"Best Overall - {best_store}"
        )

        st.header("Store Comparison")
        cols = st.columns(4)

        for col, (store, total, avg) in zip(cols, summary):
            with col:
                card(store, total, round(avg), store)


In [7]:
from pyngrok import ngrok
import subprocess
import time

# Kill previous tunnels (safety)
ngrok.kill()

# Open a fresh tunnel
public_url = ngrok.connect(8501)
print("🔗 Public URL:", public_url)

# Launch Streamlit
process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port=8501", "--server.address=0.0.0.0"]
)

# Keep the output visible
process.wait()


KeyboardInterrupt: 